# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

유사한 기능 및 모듈을 모아둔 도구인 라이브러리는 사용자가 호출을 하며 제어하는 방식인데에 반해, 
라이브러리 및 정의들을 포함하는 구조체인 프레임워크는 틀 안에 제어를 하는 흐름이 이미 존재한다는 점에서 차이가 있습니다.

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

딥러닝은 머신러닝 알고리즘 중 인공신경망을 기반으로 한 방법들 모두를 의미합니다!
따라서, 딥러닝은 머신러닝의 범주 안에 포함되는 관계입니다.

머신러닝은 사용자가 규칙을 일일이 정의하지 않아도, 자동적으로 데이터에서 규칙을 학습히는 알고리즘에 관한 분야입니다.
그 하위 개념인 딥러닝은, 계층 구조를 통해 자체적으로 배우고 결정을 내리는 인공신경망을 기반으로 한다는 특징이 있습니다.

차이점이 있자면, 
머신러닝은 데이터를 나타내는 자료구조에 따라 다양한 형태를 지닌 반면, 딥러닝은 모두 인공신경망으로 이루어져 있다는 점입니다.

### Q3. 아래의 코드에 주석 달기.

In [16]:
'''
이하 라이브러리를 불러오는 구문
- torch : facebook에서 제공하는 딥러닝 도구
- torchvision : 파이토치에서 제공하는 라이브러리. dataset 및 모듈 등을 포함.
  * torchvision.transforms => Compose() 함수 등으로 전처리 진행.
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 하나의 GPU 활용이 가능한지 torch.cuda.is_available() 함수로 확인하고, 난수 시드값을 고정하는 과정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

cuda is available


In [17]:
'''
학습에 필요한 parameter 값을 미리 정의함.
- learning_rate = 학습하는 과정 속에서 기울기의 이동 step 정도값이다.
- batch_size = 한번의 epoch를 돌때 데이터를 쪼개는데, 쪼개진 데이터의 크기를 의미한다.
- num_classes = 분류에 활용될 클래스의 개수를 의미한다.
- epochs = 전체 데이터 셋에 대해 학습을 완료하는 횟수 (forward/backward)
'''
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [18]:
# 모델 학습에 활용할 데이터 셋을 MNIST에서 불러와서 곧바로 Compose() 함수를 통해 전처리를 진행했다.
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor() 
    ])
)

# 검증용 데이터 셋을 MNIST에서 불러와서 곧바로 Compose() 함수를 통해 전처리를 진행했다.
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

In [19]:
# DataLoader를 통해 우리가 불러올 데이터 set을 불러오고, batch_size 등 옵션값을 정해준다.
# parameter 중 suffle 까지 설정하면 데이터를 무작위로 섞어서 사용할 수도 있다.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# enumerate()를 통해 TrainSet의 원소마다 tuple 생성
# example을 보기 위해 첫 원소를 뽑아서 할당해준다.
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

In [20]:
# 합성곱 신경망을 구현한 부분이다.
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() 
        # 첫 번째 합성곱 층은 1채널을 입력받아서 10개의 채널을 출력함. 5x5 정사각 컨볼루션 행렬
        # 두 번째 합성곱 층은 10채널을 입력받아서 20개의 채널을 출력함. 5x5 정사각 컨볼루션 행렬
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        
        # 과적합을 막기 위해서 Dropout을 적용했다. p값은 해당 원소가 영점조절(zeroed)될 확률.
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        
        # 커널 사이즈가 2로 설정된 MaxPooling
        self.mp = nn.MaxPool2d(2)
        
        # Fully Connected (일반 신경망) 계층을 거치면서, 출력 크기 감소 > 분류해야할 클래스 개수인 10까지 감소.
        # 초기 Input이 320인 이유 > 4 * 4 * 20 (2x2 Filter가 5x5 행렬에서 뽑아내는 feature)
        # 중간에 100은 큰 의미없는 중간값이라고 한다 (100 말고 50 정도여도 괜찮음)
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 
        
        # CNN 레이어 2개, FC 레이어 2개로 구성되어있으며, 중간중간 pooling을 진행 함.

  def forward(self, x):
        # conv > pooling > relu(활성화) 함수로 구성됨
        # pooling은 2x2 커널로 진행
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        
        # 2차원에서 1차원 형태로 변경한다 (FC 레이어에 넣기 위해)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        
        # 10개의 클래스에 대해서 확률값으로 정규화해주는 softmax 함수 사용
        return F.log_softmax(x)

In [21]:
# model > 방금 정의한 합성곱 신경망 클래스의 객체
# criterion > loss 함수로 CrossEntropyLoss() 함수를 사용했음.
# optimizer > 학습 셋을 이용해서 모델을 학습할때 실제 결과와 모델의 예측결과를 기반으로 차이를 잘 줄여줄 수 있도록 해줌.
# 찾아보니 Adam이 가장 많이 사용하는 Optimizer이라고 한다.
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [22]:
# Epoch마다 훈련을 진행한다.
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        
        # backward를 호출하기 전에, 버퍼에 누적된 변화도를 초기화.
        optimizer.zero_grad() 
        
        hypothesis = model(data)
        # 아까 정의한 loss함수를 통해 Cost를 계산한다.
        cost = criterion(hypothesis, target) 
        
        # 모델의 매개변수들에 대한 손실의 변화도를 계산한다!
        cost.backward()
        
        # 계산한 변화도에 따라 매개변수 갱신.
        optimizer.step() 
        
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

C:\Users\dksu4\AppData\Local\Temp\ipykernel_50964\2163771082.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[Epoch:    1]  cost = 0.331803977
[Epoch:    2]  cost = 0.111059882
[Epoch:    3]  cost = 0.0861956701
[Epoch:    4]  cost = 0.0733706281
[Epoch:    5]  cost = 0.0666975453


In [23]:
# model.eval() 함수를 통해 tain time과 eval time에서 수행하는 다른 작업을 수행하기 위해 switching.
# evaluation 과정에서 사용되면 안되는 layer를 off하는 함수라고 함.
# train 과정으로 돌아가야 한다면, model.train() 실행 필요.
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        
        # test 데이터를 학습시킨 모델에 넣어서 tensor를 반환받는다.
        out = model(data)
        
        # max함수를 통해, tensor에서 최대값(젤 높은 확률값)을 뽑아냄.
        preds = torch.max(out.data, 1)[1] 
        
        # 텐서끼리 비교한 후, 답과 일치하는 값의 개수를 뽑아낸다.
        total += len(target) 
        correct += (preds==target).sum().item() 
        
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

C:\Users\dksu4\AppData\Local\Temp\ipykernel_50964\2163771082.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Accuracy:  98.59 %


## 첫 정규세션 들으시느라 고생 많으셨습니다.

투빅스 화이팅.